# **GP Practice to LSOA Prescription Data Allocation**

This script performs spatial disaggregation of GP practice-level prescription data to Lower Super Output Area (LSOA) level using patient registration distribution data. The allocation methodology accounts for temporal variations in GP-patient relationships by utilizing year-specific patient registration files (2020-2025), ensuring accurate spatial attribution of prescription data over time.

The allocation process:

(1) loads annual GP patient registration data by LSOA;

(2) calculates proportional patient distribution for each GP practice across served LSOAs;

(3) allocates prescription quantities and costs proportionally based on patient distribution;

(4) aggregates allocated data to monthly LSOA-level summaries;

(5) validates allocation accuracy through conservation checks.

This approach enables fine-grained geographical analysis of diabetes prescribing patterns while accounting for the dynamic nature of primary care service boundaries and patient populations over the study period.

The resulting dataset provides estimated monthly prescription volumes and costs for each LSOA, facilitating spatial epidemiological analysis and health services research at the small-area level.

*Before executing the script, please place the following files in the current working directory: LSOA_WY.xls and all annual GP-patient registration files.*

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from tqdm import tqdm
import csv

# Data loading and preprocessing
print("Step 1: Loading data...")

# Load prescription data
prescriptions = pd.read_csv('WestYorkshire_GPDrugData_by_Practice.csv')
print(f"Prescription data shape: {prescriptions.shape}")

# Convert dates to standard format
prescriptions['date'] = pd.to_datetime(prescriptions['date'])
prescriptions['year'] = prescriptions['date'].dt.year
prescriptions['month'] = prescriptions['date'].dt.month

# Check prescription data time range
print(f"Prescription data time range: {prescriptions['date'].min()} to {prescriptions['date'].max()}")
print(f"Number of GP practices in prescription data: {prescriptions['row_id'].nunique()}")

# Load West Yorkshire LSOA data
lsoa_imd_raw = pd.read_excel('LSOA_WY.xls')
print(f"LSOA data shape: {lsoa_imd_raw.shape}")

# Get West Yorkshire LSOA list and names
lsoa_info = lsoa_imd_raw[['lsoa21cd', 'lsoa21nm']].drop_duplicates()
lsoa_info.rename(columns={'lsoa21cd': 'LSOA_CODE', 'lsoa21nm': 'LSOA_NAME'}, inplace=True)
west_yorkshire_lsoas = set(lsoa_info['LSOA_CODE'].unique())
print(f"Number of West Yorkshire LSOAs: {len(west_yorkshire_lsoas)}")

# Load and filter patient distribution data
print("\nStep 2: Loading patient distribution data...")

patient_dist_all = []

# Load data for 2020-2025
for year in range(2020, 2026):
    file_suffix = str(year)[-2:]
    filename = f'gp-reg-pat-prac-lsoa-all-{file_suffix}.csv'

    try:
        df = pd.read_csv(filename)

        # Standardize column names
        df.rename(columns={'Number of Patients': 'NUMBER_OF_PATIENTS'}, inplace=True)

        # Filter for West Yorkshire LSOAs only
        df_filtered = df[df['LSOA_CODE'].isin(west_yorkshire_lsoas)].copy()
        df_filtered['year'] = year
        patient_dist_all.append(df_filtered)
        print(f"  {year} data: {len(df_filtered)} rows (after filtering)")

        unique_gps = df_filtered['PRACTICE_CODE'].nunique()
        print(f"    - GP practices: {unique_gps}")

    except FileNotFoundError:
        print(f"  Warning: File {filename} not found")
        continue

# Merge data from all years
if patient_dist_all:
    patient_dist = pd.concat(patient_dist_all, ignore_index=True)
    print(f"\nMerged patient distribution data: {patient_dist.shape}")
else:
    print("\nError: No patient distribution data files found!")
    raise FileNotFoundError("Please ensure gp-reg-pat-prac-lsoa-all-XX.csv files are in current directory")

# Data diagnostics
print("\nStep 3: Data diagnostics...")

# Check GP code matching
prescriptions_gps = set(prescriptions['row_id'].unique())
patient_dist_gps = set(patient_dist['PRACTICE_CODE'].unique())

matched_gps = prescriptions_gps.intersection(patient_dist_gps)
unmatched_prescriptions = prescriptions_gps - patient_dist_gps
unmatched_patients = patient_dist_gps - prescriptions_gps

print(f"\nGP practice matching status:")
print(f"  GPs in prescription data: {len(prescriptions_gps)}")
print(f"  GPs in patient distribution data: {len(patient_dist_gps)}")
print(f"  Successfully matched GPs: {len(matched_gps)}")
print(f"  Unmatched GPs in prescription data: {len(unmatched_prescriptions)}")

# Calculate prescription proportion for unmatched GPs
unmatched_prescriptions_data = prescriptions[prescriptions['row_id'].isin(unmatched_prescriptions)]
total_items = prescriptions[[col for col in prescriptions.columns if '_items' in col]].sum().sum()
unmatched_items = unmatched_prescriptions_data[[col for col in prescriptions.columns if '_items' in col]].sum().sum()
if total_items > 0:
    unmatched_percentage = (unmatched_items / total_items) * 100
    print(f"  Prescription proportion from unmatched GPs: {unmatched_percentage:.2f}%")

# Calculate patient distribution proportions
print("\nStep 4: Calculating patient distribution proportions...")

# Calculate total patients per GP per year
gp_totals = patient_dist.groupby(['PRACTICE_CODE', 'year'])['NUMBER_OF_PATIENTS'].sum().reset_index()
gp_totals.rename(columns={'NUMBER_OF_PATIENTS': 'TOTAL_PATIENTS'}, inplace=True)

# Merge totals and calculate proportions
patient_dist = patient_dist.merge(gp_totals, on=['PRACTICE_CODE', 'year'])
patient_dist['PROPORTION'] = patient_dist['NUMBER_OF_PATIENTS'] / patient_dist['TOTAL_PATIENTS']

# Validate proportions
print("Proportion validation (each GP's proportions should sum to approximately 1):")
validation = patient_dist.groupby(['PRACTICE_CODE', 'year'])['PROPORTION'].sum()
print(f"  Minimum: {validation.min():.4f}")
print(f"  Maximum: {validation.max():.4f}")
print(f"  Mean: {validation.mean():.4f}")

# Allocate prescription data to LSOAs
print("\nStep 5: Allocating prescription data to LSOAs...")

# Get drug-related columns
drug_cols = [col for col in prescriptions.columns if '_items' in col or '_cost' in col]
print(f"Number of drug-related columns: {len(drug_cols)}")

# Prepare result storage
results = []
unmatched_records = []

# Process by year-month groups
for (year, month), month_data in tqdm(prescriptions.groupby(['year', 'month']),
                                      desc="Processing months"):

    # Get patient distribution data for this year
    year_dist = patient_dist[patient_dist['year'] == year]

    if len(year_dist) == 0:
        print(f"\n  Warning: No patient distribution data for {year}")
        continue

    # Allocate prescriptions for each GP
    for _, gp_row in month_data.iterrows():
        gp_code = gp_row['row_id']

        # Get all LSOAs served by this GP and their proportions
        gp_lsoas = year_dist[year_dist['PRACTICE_CODE'] == gp_code]

        if len(gp_lsoas) == 0:
            # Record unmatched GPs
            unmatched_records.append({
                'GP_CODE': gp_code,
                'GP_NAME': gp_row['row_name'],
                'year': year,
                'month': month,
                'total_items': sum(gp_row[col] for col in drug_cols if '_items' in col)
            })
            continue

        # Allocate prescriptions to each LSOA
        for _, lsoa_row in gp_lsoas.iterrows():
            result_row = {
                'LSOA_CODE': lsoa_row['LSOA_CODE'],
                'date': gp_row['date'],
                'year': year,
                'month': month,
                'GP_CODE': gp_code,
                'GP_NAME': gp_row['row_name'],
                'LSOA_PATIENTS_IN_GP': lsoa_row['NUMBER_OF_PATIENTS'],
                'GP_TOTAL_PATIENTS': lsoa_row['TOTAL_PATIENTS'],
                'PROPORTION_FROM_GP': lsoa_row['PROPORTION']
            }

            # Allocate each drug's prescription data
            for drug_col in drug_cols:
                result_row[drug_col] = gp_row[drug_col] * lsoa_row['PROPORTION']

            results.append(result_row)

# Convert to DataFrame
lsoa_prescriptions = pd.DataFrame(results)

print(f"\nInitial allocation completed, {len(lsoa_prescriptions)} rows")

# Save unmatched records
if unmatched_records:
    unmatched_df = pd.DataFrame(unmatched_records)
    unmatched_df.to_csv('unmatched_gp_records.csv', index=False, quoting=csv.QUOTE_MINIMAL)
    print(f"Unmatched GP records saved to: unmatched_gp_records.csv")
    print(f"Number of unmatched records: {len(unmatched_records)}")

# Aggregate to LSOA level
print("\nStep 6: Aggregating to LSOA level...")

# Aggregate by LSOA and date
agg_dict = {col: 'sum' for col in drug_cols}
agg_dict.update({
    'LSOA_PATIENTS_IN_GP': 'sum',  # Total LSOA population
    'GP_CODE': 'count'  # Number of GPs serving the LSOA
})

lsoa_monthly = lsoa_prescriptions.groupby(['LSOA_CODE', 'date', 'year', 'month']).agg(agg_dict).reset_index()
lsoa_monthly.rename(columns={'GP_CODE': 'NUM_GPS_SERVING',
                            'LSOA_PATIENTS_IN_GP': 'LSOA_POPULATION'}, inplace=True)

# Add LSOA names
lsoa_monthly = lsoa_monthly.merge(lsoa_info, on='LSOA_CODE', how='left')

# Data validation and diagnostics
print("\nStep 7: Data validation...")

if len(lsoa_monthly) > 0:
    # Validation 1: Check total prescription consistency
    print("Prescription total validation:")
    for drug_col in drug_cols[:2]:  # Check first two drugs as example
        original_total = prescriptions[drug_col].sum()
        allocated_total = lsoa_monthly[drug_col].sum()
        if original_total > 0:
            diff_pct = abs(original_total - allocated_total) / original_total * 100
            print(f"  {drug_col}: Original={original_total:.2f}, Allocated={allocated_total:.2f}, Difference={diff_pct:.2f}%")

    # Validation 2: Check data completeness
    print(f"\nData coverage:")
    print(f"  Number of LSOAs: {lsoa_monthly['LSOA_CODE'].nunique()}")
    print(f"  Time range: {lsoa_monthly['date'].min()} to {lsoa_monthly['date'].max()}")
    print(f"  Number of months: {lsoa_monthly['date'].nunique()}")

    # Check data by year
    print("\nYearly data summary:")
    yearly_summary = lsoa_monthly.groupby('year').agg({
        'LSOA_CODE': 'nunique',
        'Short-acting insulins_items': 'sum',
        'LSOA_POPULATION': 'mean'
    })
    print(yearly_summary)

    # Check for LSOAs with no prescription data
    empty_lsoas = lsoa_monthly.groupby('LSOA_CODE').agg({
        col: 'sum' for col in drug_cols if '_items' in col
    })
    empty_lsoas['total_items'] = empty_lsoas.sum(axis=1)
    num_empty = (empty_lsoas['total_items'] == 0).sum()
    print(f"\nNumber of LSOAs with no prescription data: {num_empty}")

# Save results
print("\nStep 8: Saving results...")

if len(lsoa_monthly) > 0:
    # Sort by date
    lsoa_monthly.sort_values(['LSOA_CODE', 'date'], inplace=True)

    # Ensure correct data types for numeric columns
    for col in drug_cols + ['LSOA_POPULATION']:
        lsoa_monthly[col] = pd.to_numeric(lsoa_monthly[col], errors='coerce')

    # Save as CSV with proper formatting
    output_filename = 'WestYorkshire_Diabetes_Prescriptions_by_LSOA_Monthly.csv'
    lsoa_monthly.to_csv(output_filename, index=False, quoting=csv.QUOTE_MINIMAL, float_format='%.6f')
    print(f"Data saved to: {output_filename}")

    # Show data preview
    print("\nData preview:")
    print(lsoa_monthly.head())
    print("\nData shape:", lsoa_monthly.shape)
    print("\nColumn names:", list(lsoa_monthly.columns))

    # Create data dictionary
    data_dict = {
        'LSOA_CODE': 'LSOA code',
        'LSOA_NAME': 'LSOA name',
        'date': 'Date',
        'year': 'Year',
        'month': 'Month',
        'LSOA_POPULATION': 'Total LSOA population (sum of patients from all GPs serving it)',
        'NUM_GPS_SERVING': 'Number of GP practices serving this LSOA',
        'Short-acting insulins_items': 'Number of short-acting insulin prescriptions',
        'Short-acting insulins_actual_cost': 'Actual cost of short-acting insulin',
        'Intermediate and long-acting insulins_items': 'Number of intermediate and long-acting insulin prescriptions',
        'Intermediate and long-acting insulins_actual_cost': 'Actual cost of intermediate and long-acting insulin',
        'Sulfonylureas_items': 'Number of sulfonylurea prescriptions',
        'Sulfonylureas_actual_cost': 'Actual cost of sulfonylureas',
        'Biguanides_items': 'Number of biguanide prescriptions (e.g., metformin)',
        'Biguanides_actual_cost': 'Actual cost of biguanides',
        'Other antidiabetic drugs_items': 'Number of other antidiabetic drug prescriptions',
        'Other antidiabetic drugs_actual_cost': 'Actual cost of other antidiabetic drugs',
        'Treatment of hypoglycaemia_items': 'Number of hypoglycemia treatment prescriptions',
        'Treatment of hypoglycaemia_actual_cost': 'Actual cost of hypoglycemia treatments',
        'Diabetic diagnostic and monitoring agents_items': 'Number of diabetic diagnostic and monitoring prescriptions',
        'Diabetic diagnostic and monitoring agents_actual_cost': 'Actual cost of diabetic diagnostic and monitoring agents'
    }

    # Save data dictionary
    with open('data_dictionary.txt', 'w', encoding='utf-8') as f:
        f.write("Data Dictionary - West Yorkshire Diabetes Prescriptions LSOA Monthly Data\n")
        f.write("="*70 + "\n\n")
        for col, desc in data_dict.items():
            f.write(f"{col}: {desc}\n")

    print("\nProcessing completed!")
    print(f"Output file: {output_filename}")
    print("Data dictionary: data_dictionary.txt")
    if unmatched_records:
        print("Unmatched GP records: unmatched_gp_records.csv")
else:
    print("\nWarning: No data generated, please check input files!")